# Liquid Liquid Equilibrium (LLE)

Phase stability plays a key role during equilibrium computation when dealing with more than two liquid phases. For this purpose the following modified multiphase Rachford-Rice mass balance has been proposed by [Gupta et al.](https://www.sciencedirect.com/science/article/pii/037838129180021M):


$$ \sum_{i=1}^c \frac{z_i (K_{ik} \exp{\theta_k}-1)}{1+ \sum\limits^{\pi}_{\substack{j=1 \\ j \neq r}}{\psi_j (K_{ij}} \exp{\theta_j} -1)} = 0 \qquad k = 1,..., \pi,  k \neq r $$

Subject to:

$$ \psi_k \theta_k = 0 $$

In this system of equations, $z_i$ represents the global composition of the component $i$,  $ K_{ij} = x_{ij}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ij} $ is the constant equilibrium of component $i$ in phase $j$ respect to the reference phase $r$, and $\psi_j$ and $\theta_j$ are the phase fraction and stability variable of the phase $j$.  

The solution strategy is similar to the classic isothermal isobaric two-phase flash. First, a reference phase must be selected, this phase is considered stable during the procedure. In an inner loop, the system of equations is solved using multidimensional Newton's method for phase fractions and stability variables and then compositions are updated in an outer loop using accelerated successive substitution (ASS).  Once the algorithm has converged, the stability variable gives information about the phase. If it takes a value of zero the phase is stable and if it is positive the phase is not.  The proposed successive substitution method can be slow, if that is the case the algorithm attempts to minimize Gibbs Free energy of the system. This procedure also ensures stable solutions and is solved using SciPy's functions.

$$ min \, {G} = \sum_{k=1}^\pi \sum_{i=1}^c F_{ik} \ln \hat{f}_{ik}  $$

This notebook shows the solution of liquid-liquid equilibrium using the ``lle`` function. This function incorporates the algorithm described above. To start, the required functions are imported.

In [4]:
import numpy as np
from phasepy import component, mixture, virialgamma, unifac
from phasepy.equilibrium import lle

The LLE calculation for the mixture of water and mtbe will be exemplified. First the mixture and its interaction parameters are set up. 

In [5]:
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

mix = mixture(water, mtbe)
# or
mix = water + mtbe

mix.unifac()
eos = virialgamma(mix, actmodel='unifac')
T = 320.0
P = 1.01
Z = np.array([0.5, 0.5])
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])
lle(x0, w0, Z, T, P, eos, full_output=True)

           T: 320.0
           P: 1.01
 error_outer: 2.9035040569607052e-09
 error_inner: 1.0073726451407202e-10
        iter: 7
        beta: array([0.53009228, 0.46990772])
       tetha: array([0.])
           X: array([[0.05876935, 0.94123065],
       [0.99774233, 0.00225767]])
           v: [None, None]
      states: ['L', 'L']

-----
Initial guesses for the phase compositions can be obtained using the ``lle_init`` function. This is shown below.

In [6]:
from phasepy.equilibrium import lle_init
from phasepy import preos 
eos = preos(mix, 'mhv_unifac')
T = 320.0
P = 1.01
z = np.array([0.5, 0.5])
lle_init(z, T, P, eos)

(array([0.99868736, 0.00131264]), array([0.11778925, 0.88221075]))

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```